In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
% cd /content/drive/MyDrive/NNFL_Assignment_2/

/content/drive/MyDrive/nnfl/assign2


In [ ]:
! pwd 

/content


In [ ]:
! ls 

drive  sample_data


In [ ]:
! sudo unzip -q nnfl-2021-assignment-2.zip

In [ ]:
import csv
import time
import gc
import numpy as np
import cv2
import sys
import matplotlib.pyplot as plt

In [ ]:
!ls

drive  sample_data


# Loading Data

In [ ]:
# Loading Names and Labels
s = time.time()

train_vid_names = []
train_vid_label = []

train_csv = open('/content/drive/MyDrive/NNFL_Assignment_2/train.csv')
reader = csv.DictReader(train_csv)

for row in reader:
    train_vid_names.append(row['File'])
    train_vid_label.append([1,0] if row['True_Label']=='fire' else [0,1])
    
train_csv.close()
train_vid_label = np.array(train_vid_label)
e = time.time()
print(e-s)
gc.collect()

0.012621402740478516


55

In [ ]:
print(len(train_vid_names))

120


In [ ]:
from tqdm.auto import tqdm

In [ ]:
max_frames = 150
frame_size = (128, 128)

frames = np.zeros((len(train_vid_names), max_frames, frame_size[0], frame_size[1], 3), dtype=np.float32)
frame_count = []
train_folder = '/content/drive/MyDrive/NNFL_Assignment_2/train/'

print(sys.getsizeof(frames))
gc.collect()

s = time.time()

for i in tqdm(range(len(train_vid_names))):
    
    vid = cv2.VideoCapture(train_folder+train_vid_names[i])
    
    
    # Load full video in temp
    temp = [] 
    ret, frame = vid.read()
    while ret:
        temp.append(cv2.resize(frame,frame_size))
        ret, frame = vid.read()
     
    length = len(temp)
    
    if length < max_frames:
        
        pads_required = max_frames % length
        each_frame_multiplicity = max_frames // length
        for _ in range(len(temp)):
            for __ in range(each_frame_multiplicity):
                frames[i][pads_required + _*each_frame_multiplicity + __] = temp[_]
    else:
        
        jump = length / max_frames
        for _ in range(max_frames):
            frames[i][_] = temp[int(_*jump)]
    # print(i)
    frame_count.append(frames[i].shape)
    gc.collect()

e = time.time()
print(e-s)

gc.collect()

3538944160


  0%|          | 0/120 [00:00<?, ?it/s]

126.21710109710693


19

In [ ]:
gc.collect()

147

# Normalization

In [ ]:
# means = []
# stds = []
# for i in range(3):
#   means.append(np.mean(frames[:,:,:,:,i]))
#   stds.append(np.std(frames[:, :, :, :, i]))

In [ ]:
# means = np.asarray(means).reshape(1, 1, 1, 1, -1)
# means.shape

In [ ]:
# mean = np.mean(frames, axis=-1).reshape([*frames.shape[:-1], 1])
# mean = np.mean(frames, axis=-1).reshape
# std = np.std(frames, axis=-1).reshape([*frames.shape[:-1], 1])
# 
# frames = (frames - mean) / (1e-5 + std)

# The Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

gpus = tf.config.experimental.list_physical_devices('GPU')
for i in gpus:
    print(i)
    tf.config.experimental.set_memory_growth(i, True)

In [ ]:
gc.collect()

50

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.Input(shape = (max_frames, frame_size[0], frame_size[1], 3)))
model.add(layers.TimeDistributed(layers.Conv2D(12, (3, 3), activation='relu')))
model.add(layers.TimeDistributed(layers.Conv2D(25, (3, 3), activation='relu')))
model.add(layers.TimeDistributed(layers.Conv2D(50, (3, 3), activation='relu')))
model.add(layers.TimeDistributed(layers.MaxPool2D()))
model.add(layers.TimeDistributed(layers.Conv2D(75, (3, 3), activation='relu')))
model.add(layers.TimeDistributed(layers.MaxPool2D()))
model.add(layers.TimeDistributed(layers.Conv2D(100, (3, 3), activation='relu')))
model.add(layers.TimeDistributed(layers.MaxPool2D()))
model.add(layers.TimeDistributed(layers.Conv2D(128, (3, 3), activation='relu')))
model.add(layers.TimeDistributed(layers.MaxPool2D()))
model.add(layers.TimeDistributed(layers.Conv2D(256, (3, 3), activation='relu')))
model.add(layers.TimeDistributed(layers.MaxPool2D()))
model.add(layers.TimeDistributed(layers.Flatten()))
model.add(layers.LSTM(25, return_sequences=False))
model.add(layers.Dense(75, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(25, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 150, 126, 126, 12  336      
 ibuted)                     )                                   
                                                                 
 time_distributed_1 (TimeDis  (None, 150, 124, 124, 25  2725     
 tributed)                   )                                   
                                                                 
 time_distributed_2 (TimeDis  (None, 150, 122, 122, 50  11300    
 tributed)                   )                                   
                                                                 
 time_distributed_3 (TimeDis  (None, 150, 61, 61, 50)  0         
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 150, 59, 59, 75)  3

# Without changing lr

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(frames, train_vid_label, test_size=0.2, shuffle=True)

# the_model.fit(x=x_train, y=y_train, batch_size=64, epochs=100, validation_data=(x_val, y_val))

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6),
                loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(x=x_train, y=y_train, batch_size = 1, epochs = 25, validation_data=(x_valid, y_valid))

Epoch 1/25
96/96 [==============================] - 1235s 13s/step - loss: 0.6860 - accuracy: 0.5625 - val_loss: 0.7029 - val_accuracy: 0.4583
Epoch 2/25
96/96 [==============================] - 1232s 13s/step - loss: 0.6808 - accuracy: 0.5625 - val_loss: 0.6901 - val_accuracy: 0.4583
Epoch 3/25
26/96 [=======>......................] - ETA: 13:38 - loss: 0.6568 - accuracy: 0.6538

In [ ]:
# del model
# del history

# Plots

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'], loc = 'lower right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'validation'], loc = 'upper right')
plt.show()

gc.collect()

# Generating Submission.csv

In [ ]:
s = time.time()
output = model.predict(frames_t)
e = time.time()
print(e-s)

test_vid_label = []
for i in output:
    test_vid_label.append('fire' if i[0]>0.5 else 'not_fire')

2.9528274536132812


In [ ]:
! touch Submission.csv

open('Submission.csv', 'w').close()

output = open('Submission.csv', 'w')

header = ['File', 'Label']
writer = csv.writer(output)
writer.writerow(header)

for i in range(len(test_vid_names)):
    writer.writerow([test_vid_names[i],test_vid_label[i]])

output.close()